In [1]:
import pandas as pd
import re
import PyPDF2
import os
import json
import copy
import numpy as np
import tabula as tb

In [45]:
pdf_path = "2015_house_pdfs/Pelosi_Nancy_10010857.pdf"
pdf_path_joe = "2015_house_pdfs/Wilson_Joe_10012512.pdf"
pdf_to_empty = "2015_house_pdfs/Pelosi_Nancy_20002351.pdf"
pdf_to_apple = "2018_house_pdfs/Apple_Gary Winston_10022824.pdf"
pdf_to_young = "2017_house_pdfs/Young_Stephen Robert Neale_10018941.pdf"
pdf_to_wass = "2019_house_pdfs/Wasserman Schultz_Debbie_10037015.pdf"
pdf_to_nancy = "2019_house_pdfs/Pelosi_Nancy_10035243.pdf"
pdf_to_beyer =  "2018_house_pdfs/Beyer_Donald Sternoff_10026686.pdf"
pdf_to_low = "2018_house_pdfs/Lowenthal_Alan S._10025964.pdf"

list_of_pdf_paths = (pdf_path, pdf_path_joe, pdf_to_nancy, pdf_to_wass, pdf_to_low)

In [3]:
description_regex = r"^description: (.+)$"
stock_regex = r"^\s*(?!description:|location:)\s*(.*)\s*\((?!one|two)([\w | .]{1,5})\).*$"
bounds_regex = r"^[a-zA-Z]*\s*\$?(?!\d{1,4}\/)([\d]+,?[\d]+)\s*-?\s*\$?([\d]*,?\d*).*$"
time_regex = r"^[a-zA-z]*\s*(\d{1,2}\/\d{1,2}\/\d{2,4})\s*$"

transaction_type_regex = r"\s*([SPsp])\s*$"

In [4]:
pdf_to_empty = "2015_house_pdfs/Pelosi_Nancy_20002351.pdf"
if os.stat(pdf_to_empty).st_size < 2000:
    print("smaller")
else:
    print("larger")

smaller


In [5]:
def create_template(file_path):
    file = open(file_path, 'rb')
    readpdf = PyPDF2.PdfFileReader(file)
    total_pages = readpdf.numPages
    with open("pdf_reduced_template.tabula-template.json", "r") as jsonTemplate:
        data = json.load(jsonTemplate)
        page_template = data[3]
        # print(page_template)
        if total_pages == 1:
            data = data[:-1]
        else:
            for page_num in range(2, total_pages):
                tmp_page_template = copy.copy(page_template)
                tmp_page_template['page'] = page_num + 1
                # print(tmp_page_template)
                data.append(tmp_page_template)
            
        with open("pdf_tmp_template.tabula-template.json", 'w') as outfile:
            json.dump(data, outfile)
                
            
            

In [6]:
def scrape_pdf(file):
    create_template(file)
    template_path = 'pdf_tmp_template.tabula-template.json'
    return tb.read_pdf_with_template(file, template_path, stream=True)

In [7]:
db = scrape_pdf(pdf_path)

In [8]:
db_joe = scrape_pdf(pdf_path_joe)

In [9]:
db_apple = scrape_pdf(pdf_to_apple)

In [10]:
db_young = scrape_pdf(pdf_to_young)

In [11]:
db_wass = scrape_pdf(pdf_to_wass)

In [12]:
db_nancy = scrape_pdf(pdf_to_nancy)

In [13]:
pdf_db = pd.DataFrame(columns=['Member Name', 'Member District', 'Company', 'Ticker', 'Type', 'Date',
                      'Value Lower Bound','Value Upper Bound', 'Description', 'Link'])

In [14]:
db[7]

,asset owner,value of asset,income,Unnamed: 0,income tx. >
0,NaN,NaN,type(s),NaN,"$1,000?"
1,NaN,NaN,NaN,NaN,cdfeg
2,Account,NaN,NaN,NaN,NaN
3,"DESCRIPTION: San Francisco, CA",NaN,NaN,NaN,NaN
4,Union Bank of California - Pelosi Trust Checki...,"$1 - $1,000",None,NaN,ecgdf
5,Account,NaN,NaN,NaN,NaN
6,"DESCRIPTION: San Francisco, CA",NaN,NaN,NaN,NaN
7,United Football League SP,"$5,000,001 -",None,NaN,fcedg
8,NaN,"$25,000,000",NaN,NaN,NaN
9,"LOCATION: Jacksonville/Duval, FL, US",NaN,NaN,NaN,NaN


In [15]:
"schedule b: transactions" in db[7]["asset owner"].str.lower().values

True

In [16]:
db[8].iloc[:, 0]

0                                                   NaN
1                                                   NaN
2                              Salesforce.com Inc (CRM)
3                                                   NaN
4     DESCRIPTION: Contribution of 1,084 shares held...
5                                             accounts.
6                                         Visa Inc. (V)
7                                                   NaN
8                     DESCRIPTION: Sale of 5,000 Shares
9                                         Visa Inc. (V)
10                                                  NaN
11    DESCRIPTION: Contribution of 2,000 shares held...
12                                            accounts.
13                            ScHeDule c: earneD income
14                                          Source type
15             Financial Leasing Services Spouse Salary
16           Fairfield Associates Spouse Management Fee
17    Forty-Five Belden Corporation Spouse Manag

In [17]:
index = db[6].index[db[6]["asset owner"].str.lower() == "schedule b: transactions"].tolist()
index



[]

In [18]:
def find_start(df):
    for i in range(len(df)):
        index = df[i].index[df[i].iloc[:, 0].str.lower() == "schedule b: transactions"].tolist()
        if index != []:
            return i, index[0]

In [19]:
find_start(db_joe)

(3, 11)

In [20]:
db[0].iloc[2, 1]

'CA12'

In [21]:
pd.to_datetime(db[8].iloc[9, 1])

Timestamp('2015-12-30 00:00:00')

In [22]:
db_nancy[3]

,asset owner,value of asset,Unnamed: 0,income,income.1,tx. >
0,NaN,NaN,NaN,type(s),NaN,"$1,000?"
1,Aristotle LLC [OL] SP,"$15,001 - $50,000",NaN,None,NaN,dcfeg
2,"LOCATION: San Francisco/San Francisco, CA, US",NaN,NaN,NaN,NaN,NaN
3,DESCRIPTION: Ltd. Partnership Investment in Ar...,NaN,NaN,NaN,NaN,NaN
4,AT&T Inc. (T) [ST] SP,"$250,001 -",NaN,Dividends,"$15,001 - $50,000",ecdgf
5,NaN,"$500,000",NaN,NaN,NaN,NaN
6,Auberge du Soleil [OL] SP,"$5,000,001 -",NaN,Partnership Loss,"$5,001 - $15,000",gdecf
7,NaN,"$25,000,000",NaN,NaN,NaN,NaN
8,"LOCATION: Rutherford/Napa, CA, US",NaN,NaN,NaN,NaN,NaN
9,DESCRIPTION: Ltd. Partnership Investment in Re...,NaN,NaN,NaN,NaN,NaN


In [23]:
def get_column_for_regex(df, df_index, regex, start_row=0):
    number_of_columns = len(df[df_index].columns)
    number_of_rows = len(df[df_index])
    column_number = 1
    row_number = start_row
    regex_result = re.match(regex, str(df[df_index].iloc[row_number, column_number]))
    while regex_result is None and column_number < number_of_columns:
        if row_number < (number_of_rows - 1):
            # print(row_number, column_number)
            row_number += 1
        elif column_number < (number_of_columns - 1):
            # print(row_number, column_number)
            row_number = start_row
            column_number += 1
        else:
            break
        
        regex_result = re.match(regex, str(df[df_index].iloc[row_number, column_number]))
    
    if regex_result is None:
        print(f"exception raised here: {df_index}, {row_number}, {column_number}")
        raise Exception("Column not found for given regex")
    
    return column_number
        

In [24]:
db[8]

,asset owner,Date,tx.,amount,cap.
0,NaN,NaN,type,NaN,gains >
1,NaN,NaN,NaN,NaN,$200?
2,Salesforce.com Inc (CRM),12/30/2015,S,"$50,001 -",dbgfce
3,NaN,NaN,(partial),"$100,000",NaN
4,"DESCRIPTION: Contribution of 1,084 shares held...",NaN,NaN,NaN,NaN
5,accounts.,NaN,NaN,NaN,NaN
6,Visa Inc. (V),12/29/2015,S,"$250,001 -",ecgdbf
7,NaN,NaN,(partial),"$500,000",NaN
8,"DESCRIPTION: Sale of 5,000 Shares",NaN,NaN,NaN,NaN
9,Visa Inc. (V),12/30/2015,S,"$100,001 -",cdgefb


In [25]:
str(db[8].iloc[24, 1])

'Helena, CA'

In [26]:
get_column_for_regex(db, 8, bounds_regex, 0)

3

In [69]:
def generate_entry(df, df_index, row, regex_result, new_db_page):
    
    description_regex = r"^description: (.+)$"
    stock_regex = r"^\s*(?!description:|location:)\s*(.*)\s*\((?!one|two)([\w | .]{1,5})\).*$"
    bounds_regex = r"^[a-zA-Z]*\s*\$?(?!\d{1,4}\/)([\d]+,?[\d]+)\s*-?\s*\$?([\d]*,?\d*).*$"
    time_regex = r"^[a-zA-z]*\s*(\d{1,2}\/\d{1,2}\/\d{2,4})\s*$"
    transaction_type_regex = r"\s*([SPsp])\s*$"
    
    row_shift_flag = False
    df_index_shift_flag = False
    prev_row = row
    prev_df_index = df_index
        
    company_name = regex_result[1].strip()
    ticker = regex_result[2].upper()
    if new_db_page:
        column_time = get_column_for_regex(df, df_index, time_regex)
    else:
        column_time = get_column_for_regex(df, df_index, time_regex, row)
    
    time_regex_result = re.match(time_regex, str(df[df_index].iloc[row, column_time]))
    
    while time_regex_result is None:
        if row == 0:
            df_index -= 1
            row = len(df[df_index]) - 1
            column_time = get_column_for_regex(df, df_index, time_regex, row-1)
        else:
            row -= 1
        time_regex_result = re.match(time_regex, str(df[df_index].iloc[row, column_time]))
    print(df_index, row)
    """
    
    if time_regex_result is None:
        row_shift_flag = True
        row -= 1
        
    time_regex_result = re.match(time_regex, str(df[df_index].iloc[row, column_time]))
    if time_regex_result is None:
        df_index -= 1
        prev_row = row
        row = len(df[df_index]) - 1
        df_index_shift_flag = True
        
        column_time = get_column_for_regex(df, df_index, time_regex, row-1)
        
        time_regex_result = re.match(time_regex, str(df[df_index].iloc[row, column_time]))
        
    """
        
    print(df_index, row, column_time)
    owner_date = pd.to_datetime(time_regex_result[1])
    
    if new_db_page:
        transaction_bounds = get_column_for_regex(df, df_index, transaction_type_regex)
    else:
        transaction_bounds = get_column_for_regex(df, df_index, transaction_type_regex, row)
        
    transaction_type = df[df_index].iloc[row, transaction_bounds]
    
    if new_db_page:
        column_bounds = get_column_for_regex(df, df_index, bounds_regex)
    else:
        column_bounds = get_column_for_regex(df, df_index, bounds_regex, row)
    bounds_regex_result = re.match(bounds_regex, df[df_index].iloc[row, column_bounds])


    # print(df[df_index].iloc[row, column_bounds])
    lower_bound = bounds_regex_result[1]
    upper_bound = ''

    if bounds_regex_result[2] == '':
        if row == len(df[df_index]) - 1:
            upper_bound = lower_bound
        else:
            regex_result = re.match(bounds_regex, str(df[df_index].iloc[row+1, column_bounds]))
            # print(df_index, row, column_bounds)
            if regex_result is None:
                upper_bound = lower_bound
            else:
                upper_bound = regex_result[1]
                

    else:
        upper_bound = bounds_regex_result[2]
        
    row = prev_row
    df_index = prev_df_index
    
    
    
    description = None
    
    location_regex = r"^location: (.+)$"
        
    row += 1
    max_row_not_hit = True
    if row >= len(df[df_index]) - 1:
        max_row_not_hit = False
    else:
        line_info = str(df[df_index].iloc[row, 0]).lower()
        location_regex_result = re.match(location_regex, line_info)
        
    
    while max_row_not_hit and (line_info == 'nan' or location_regex_result is not None):
        if len(df[df_index])-1 == row:
            max_row_not_hit = False
        else:
            row += 1
            line_info = str(df[df_index].iloc[row, 0]).lower()
            location_regex_result = re.match(location_regex, line_info)
    
    if max_row_not_hit:
        description_regex_result = re.match(description_regex, line_info)
        if description_regex_result is not None:
            description = description_regex_result[1]
            
    
        
    """
    while exit and potential_new_stock is None:
        regex_result = re.match(description_regex, str(df[df_index].iloc[row, 0]).lower())
        if regex_result is not None:
            description = regex_result[1]
            break
            
        if len(df[df_index])-1 == row:
            exit = False
        else:
            row += 1
            stock_info = str(df[df_index].iloc[row, 0]).lower()
            potential_new_stock = re.match(stock_regex, stock_info)
    """
        
    return {'Member Name': np.nan,
            'Member District': np.nan,
            'Company': company_name,
            'Ticker': ticker,
            'Type': transaction_type,
            'Date': owner_date,
            'Value Lower Bound' : lower_bound,
            'Value Upper Bound' : upper_bound,
            'Description' : description,
            'Link' : np.nan}, row
        

In [28]:
res = re.match(stock_regex, str(db[7].iloc[31, 0]).lower())
test_entry, row = generate_entry(db, 7, 31, res, False)
test_entry

7 31
7 31 1


{'Member Name': nan,
 'Member District': nan,
 'Company': 'hertz global holdings, inc',
 'Ticker': 'HTZ',
 'Type': 'P',
 'Date': Timestamp('2015-07-16 00:00:00'),
 'Value Lower Bound': '100,001',
 'Value Upper Bound': '250,000',
 'Description': 'purchase of 258 call options with a strike price of $14 and an expiration date of 1/15/16',
 'Link': nan}

In [29]:
pdf_db = pd.concat([pdf_db, pd.Series(test_entry).to_frame().T], ignore_index=True)
pdf_db

,Member Name,Member District,Company,Ticker,Type,Date,Value Lower Bound,Value Upper Bound,Description,Link
0,NaN,NaN,"hertz global holdings, inc",HTZ,P,2015-07-16 00:00:00,"100,001","250,000",purchase of 258 call options with a strike pri...,NaN


In [30]:
def process_dataframe(df, link):
    
    pdf_db = pd.DataFrame(columns=['Member Name', 'Member District', 'Company', 'Ticker', 'Type', 'Date',
                      'Value Lower Bound','Value Upper Bound', 'Description', 'Link'])
    
    # description_regex = r"^description: (.+)$"
    stock_regex = r"^\s*(?!description:|location:)\s*(.*)\s*\((?!one|two)([\w | .]{1,5})\).*$"
    # bounds_regex = r"^[a-zA-Z]*\s*\$?([\d,]+)\s*-?\s*\$?([\d,]*).*$"
    honorifics_regex = r"^(hon.?|mr.?|miss|mrs.?|ms.?|dr.?|professor.?|gen.?|)\s*(.+)$"
    # NEED TO ADD TIME REGEX
    
    new_db_page = False

    name_regex_result = re.match(honorifics_regex, df[0].iloc[0, 1].lower())
    name = name_regex_result[2].strip()
                             
    district = df[0].iloc[2, 1]            
    find_start_result = find_start(df)                         
    if find_start_result is None:
        return pdf_db
    
    df_index, row = find_start_result
    
    
    
    # row += 1
    stock_info = str(df[df_index].iloc[row, 0]).lower()

    while stock_info != "schedule c: earned income" and stock_info != "schedule d: liabilities":
        regex_result = re.match(stock_regex, stock_info)
        if regex_result is not None:
            result_dict, new_row = generate_entry(df, df_index, row, regex_result, new_db_page)
            result_dict['Member Name'] = name
            result_dict['Member District'] = district
            result_dict['Link'] = link
            pdf_db = pd.concat([pdf_db, pd.Series(result_dict).to_frame().T], ignore_index=True)
            
            
        if len(df[df_index].index)-1 == row:
            row = 0
            df_index += 1
            print("HELLO")
            new_db_page = True
        else:
            row += 1
        # print(stock_info)
        stock_info = str(df[df_index].iloc[row, 0]).lower()
    
    return pdf_db

In [65]:
processed_nancy = process_dataframe(db_nancy, None)

7 15
7 15 1
7 18
7 18 1
7 20
7 20 1
7 23
7 23 1
HELLO
8 2
8 2 1
8 5
8 5 1
8 8
8 8 1
8 11
8 11 1


In [66]:
procesed_nancy_2013 = process_dataframe(db, None)

7 29
7 29 1
7 31
7 31 1
HELLO
8 2
8 2 1
8 6
8 6 1
8 9
8 9 1


In [61]:
db[8]

,asset owner,Date,tx.,amount,cap.
0,NaN,NaN,type,NaN,gains >
1,NaN,NaN,NaN,NaN,$200?
2,Salesforce.com Inc (CRM),12/30/2015,S,"$50,001 -",dbgfce
3,NaN,NaN,(partial),"$100,000",NaN
4,"DESCRIPTION: Contribution of 1,084 shares held...",NaN,NaN,NaN,NaN
5,accounts.,NaN,NaN,NaN,NaN
6,Visa Inc. (V),12/29/2015,S,"$250,001 -",ecgdbf
7,NaN,NaN,(partial),"$500,000",NaN
8,"DESCRIPTION: Sale of 5,000 Shares",NaN,NaN,NaN,NaN
9,Visa Inc. (V),12/30/2015,S,"$100,001 -",cdgefb


In [67]:
pd.concat([procesed_nancy_2013, processed_nancy])

,Member Name,Member District,Company,Ticker,Type,Date,Value Lower Bound,Value Upper Bound,Description,Link
0,nancy pelosi,CA12,"hertz global holdings, inc",HTZ,P,2015-07-15 00:00:00,"15,001","50,000",purchase of 42 call options with a strike pric...,None
1,nancy pelosi,CA12,"hertz global holdings, inc",HTZ,P,2015-07-16 00:00:00,"100,001","250,000",purchase of 258 call options with a strike pri...,None
2,nancy pelosi,CA12,salesforce.com inc,CRM,S,2015-12-30 00:00:00,"50,001","100,000","contribution of 1,084 shares held personally t...",None
3,nancy pelosi,CA12,visa inc.,V,S,2015-12-29 00:00:00,"250,001","500,000","sale of 5,000 shares",None
4,nancy pelosi,CA12,visa inc.,V,S,2015-12-30 00:00:00,"100,001","250,000","contribution of 2,000 shares held personally t...",None
0,nancy pelosi,CA12,"amazon.com, inc.",AMZN,P,2019-07-22 00:00:00,"500,001","1,000",purchased 20 call options with a strike price ...,None
1,nancy pelosi,CA12,bf enterprises common stock new,BFET,S,2019-02-07 00:00:00,"1,001","15,000",final cash liquidation initiated by corporation.,None
2,nancy pelosi,CA12,"netflix, inc.",NFLX,P,2019-07-05 00:00:00,"250,001","500,000",purchased 20 call options with a strike price ...,None
3,nancy pelosi,CA12,"netflix, inc.",NFLX,P,2019-07-22 00:00:00,"100,001","250,000",purchased 30 call options with a strike price ...,None
4,nancy pelosi,CA12,salesforce.com inc,CRM,P,2019-06-14 00:00:00,"50,001","100,000",purchased 20 call options with a strike price ...,None


In [35]:
list_of_pdf_paths[1:]

('2015_house_pdfs/Wilson_Joe_10012512.pdf',
 '2019_house_pdfs/Pelosi_Nancy_10035243.pdf',
 '2019_house_pdfs/Wasserman Schultz_Debbie_10037015.pdf')

In [36]:
def scrap_and_process(pdf_paths_list):
    unprocessed_db = scrape_pdf(pdf_paths_list[0])
    final_db = process_dataframe(unprocessed_db, None)
    for pdf_path in pdf_paths_list[1:]:
        unprocessed_db = scrape_pdf(pdf_path)
        processed_db = process_dataframe(unprocessed_db, None)
        final_db = pd.concat([final_db, processed_db])
        
    return final_db

In [46]:
scrap_and_process(list_of_pdf_paths)

7 29
7 29 1
7 31
7 31 1
HELLO
8 2
8 2 1
8 6
8 6 1
8 9
8 9 1
7 15
7 15 1
7 18
7 18 1
7 20
7 20 1
7 23
7 23 1
HELLO
8 2
8 2 1
8 5
8 5 1
8 8
8 8 1
8 11
8 11 1
HELLO
17 2
17 2 2
17 4
17 4 2
17 6
17 6 2
17 8
17 8 2
17 10
17 10 2
17 12
17 12 2
17 14
17 14 2
17 16
17 16 2
17 18
17 18 2
17 20
17 20 2
17 22
17 22 2
17 24
17 24 2
17 26
17 26 2
17 28
17 28 2
HELLO
17 30
17 30 2
18 4
18 4 2
18 6
18 6 2
18 8
18 8 2
18 10
18 10 2
18 12
18 12 2
18 14
18 14 2
18 16
18 16 2
18 18
18 18 2
18 20
18 20 2
18 22
18 22 2
18 24
18 24 2
18 29
18 29 2
18 31
18 31 2
HELLO
19 2
19 2 1
19 4
19 4 1
19 6
19 6 1
19 8
19 8 1
19 13
19 13 1
19 15
19 15 1
19 17
19 17 1
19 19
19 19 1
19 21
19 21 1
19 23
19 23 1
19 25
19 25 1
19 27
19 27 1
19 29
19 29 1
HELLO
19 31
19 31 1
20 4
20 4 1
20 6
20 6 1
20 8
20 8 1
20 10
20 10 1
20 12
20 12 1
20 15
20 15 1
20 18
20 18 1
20 23
20 23 1
20 27
20 27 1
20 29
20 29 1
20 31
20 31 1
HELLO
20 33
20 33 1
21 4
21 4 1
21 6
21 6 1
21 8
21 8 1
21 10
21 10 1
21 17
21 17 1
21 19
21 19 1
21 21
21

,Member Name,Member District,Company,Ticker,Type,Date,Value Lower Bound,Value Upper Bound,Description,Link
0,nancy pelosi,CA12,"hertz global holdings, inc",HTZ,P,2015-07-15 00:00:00,"15,001","50,000",purchase of 42 call options with a strike pric...,None
1,nancy pelosi,CA12,"hertz global holdings, inc",HTZ,P,2015-07-16 00:00:00,"100,001","250,000",purchase of 258 call options with a strike pri...,None
2,nancy pelosi,CA12,salesforce.com inc,CRM,S,2015-12-30 00:00:00,"50,001","100,000","contribution of 1,084 shares held personally t...",None
3,nancy pelosi,CA12,visa inc.,V,S,2015-12-29 00:00:00,"250,001","500,000","sale of 5,000 shares",None
4,nancy pelosi,CA12,visa inc.,V,S,2015-12-30 00:00:00,"100,001","250,000","contribution of 2,000 shares held personally t...",None
...,...,...,...,...,...,...,...,...,...,...
191,alan s. lowenthal,CA47,universal display corporation,OLED,S,2018-12-06 00:00:00,"1,001","15,000","maturity date is 11/1/2021, interest rate is 3...",None
192,alan s. lowenthal,CA47,wex inc. common stock,WEX,S,2018-06-04 00:00:00,"1,001","15,000",None,None
193,alan s. lowenthal,CA47,wex inc. common stock,WEX,S,2018-07-23 00:00:00,"1,001","15,000",None,None
194,alan s. lowenthal,CA47,wex inc. common stock,WEX,S,2018-07-23 00:00:00,"1,001","15,000",None,None


In [38]:
tmp = scrape_pdf(pdf_to_low)


In [51]:
tmp[32]

,asset,owner,Date,tx.,amount,cap.
0,NaN,NaN,NaN,type,NaN,gains >
1,NaN,NaN,NaN,NaN,NaN,$200?
2,NaN,NaN,NaN,NaN,NaN,efdcg
3,Oracle Corporation (ORCL) [ST],NaN,NaN,(partial),NaN,NaN
4,Neuberger Berman - Traditional IRA ⇒,SP,03/1/2018,S,"$1,001 - $15,000",cefdg
5,Oracle Corporation (ORCL) [ST],NaN,NaN,NaN,NaN,NaN
6,Neuberger Berman - Traditional IRA ⇒,SP,07/20/2018,P,"$1,001 - $15,000",NaN
7,"Red Hat, Inc. (RHT) [ST]",NaN,NaN,NaN,NaN,NaN
8,Neuberger Berman - Traditional IRA ⇒,SP,08/29/2018,P,"$1,001 - $15,000",NaN
9,"Red Hat, Inc. (RHT) [ST]",NaN,NaN,NaN,NaN,NaN


In [70]:
process_dataframe(tmp, None)

HELLO
17 2
17 2 2
17 4
17 4 2
17 6
17 6 2
17 8
17 8 2
17 10
17 10 2
17 12
17 12 2
17 14
17 14 2
17 16
17 16 2
17 18
17 18 2
17 20
17 20 2
17 22
17 22 2
17 24
17 24 2
17 26
17 26 2
17 28
17 28 2
HELLO
17 30
17 30 2
18 4
18 4 2
18 6
18 6 2
18 8
18 8 2
18 10
18 10 2
18 12
18 12 2
18 14
18 14 2
18 16
18 16 2
18 18
18 18 2
18 20
18 20 2
18 22
18 22 2
18 24
18 24 2
18 29
18 29 2
18 31
18 31 2
HELLO
19 2
19 2 1
19 4
19 4 1
19 6
19 6 1
19 8
19 8 1
19 13
19 13 1
19 15
19 15 1
19 17
19 17 1
19 19
19 19 1
19 21
19 21 1
19 23
19 23 1
19 25
19 25 1
19 27
19 27 1
19 29
19 29 1
HELLO
19 31
19 31 1
20 4
20 4 1
20 6
20 6 1
20 8
20 8 1
20 10
20 10 1
20 12
20 12 1
20 15
20 15 1
20 18
20 18 1
20 23
20 23 1
20 27
20 27 1
20 29
20 29 1
20 31
20 31 1
HELLO
20 33
20 33 1
21 4
21 4 1
21 6
21 6 1
21 8
21 8 1
21 10
21 10 1
21 17
21 17 1
21 19
21 19 1
21 21
21 21 1
21 23
21 23 1
21 25
21 25 1
21 27
21 27 1
21 29
21 29 1
21 31
21 31 1
HELLO
22 2
22 2 2
22 4
22 4 2
22 6
22 6 2
22 8
22 8 2
22 10
22 10 2
22 13
22 13 

,Member Name,Member District,Company,Ticker,Type,Date,Value Lower Bound,Value Upper Bound,Description,Link
0,alan s. lowenthal,CA47,"activision blizzard, inc",ATVI,P,2018-10-24 00:00:00,207,207,None,None
1,alan s. lowenthal,CA47,"air products and chemicals, inc.",APD,S,2018-07-19 00:00:00,464,464,None,None
2,alan s. lowenthal,CA47,"air products and chemicals, inc.",APD,S,2018-07-20 00:00:00,"1,001","15,000",None,None
3,alan s. lowenthal,CA47,"air products and chemicals, inc.",APD,P,2018-12-03 00:00:00,826,826,None,None
4,alan s. lowenthal,CA47,"air products and chemicals, inc.",APD,P,2018-12-04 00:00:00,658,658,None,None
...,...,...,...,...,...,...,...,...,...,...
191,alan s. lowenthal,CA47,universal display corporation,OLED,S,2018-12-06 00:00:00,"1,001","15,000",None,None
192,alan s. lowenthal,CA47,wex inc. common stock,WEX,S,2018-06-04 00:00:00,"1,001","15,000",None,None
193,alan s. lowenthal,CA47,wex inc. common stock,WEX,S,2018-07-23 00:00:00,"1,001","15,000",None,None
194,alan s. lowenthal,CA47,wex inc. common stock,WEX,S,2018-07-23 00:00:00,"1,001","15,000",None,None
